In [1]:
import pandas as pd
from IPython.display import display


In [2]:
'''
   run_bash_cmdning django script and jupyter in this case have
   different working directories
   so this is done to make sure that the data path is correct for both
'''
import os
cwd = os.getcwd()
curr_dir = cwd.split(os.path.sep)[-1]
IS_NOTEBOOK = curr_dir == 'scripts'

if IS_NOTEBOOK: 
   os.chdir('../')

NEW_CWD = os.getcwd()
DATA_PATH = os.path.join(NEW_CWD, 'data','all-states-history.csv')
print('cwd before checking: ', cwd)
print('DATA_PATH: ', DATA_PATH)
print('cwd now: ', NEW_CWD)
print('IS_NOTEBOOK: ', IS_NOTEBOOK)

cwd before checking:  f:\Projects\geo-covid-backend\scripts
DATA_PATH:  f:\Projects\geo-covid-backend\data\all-states-history.csv
cwd now:  f:\Projects\geo-covid-backend
IS_NOTEBOOK:  True


In [3]:
''' Activate django env if is in notebook '''

if IS_NOTEBOOK:
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geo_covid.settings')
    import django
    django.setup()

In [4]:
'''
    Meta functions
'''
import subprocess

def decode_output(str):
    try:
        return str.decode('utf-8')
    except UnicodeDecodeError as e:
        return str.decode('latin-1')
def run_bash_cmd(command: str):
    print("Exec: ", command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    if output: print('Out: ', decode_output(output))
    if error: print('Err: ', decode_output(error))

In [5]:
'''
    Helper functions
'''
import shutil
import re

CASE_CONVERT_PATTERN = re.compile(r'(?<!^)(?=[A-Z])')
def camel_case_to_snake_case(string):
    return CASE_CONVERT_PATTERN.sub('_', string).lower()

def clear_migrations():
    try:
        shutil.rmtree("./us_covid_api/migrations/")
    except BaseException as e:
        print('Fail to clear migration: ',e)

def clean_cache():
    run_bash_cmd('python manage.py clear_cache')
    run_bash_cmd('python manage.py clean_pyc')
    run_bash_cmd(f'pyclean {NEW_CWD}')

def drop_database():
    from pymongo import MongoClient
    client = MongoClient('localhost', 27017)
    client.drop_database('geo-covid')

def clean_and_remigrate_db():
    run_bash_cmd("python manage.py flush --noinput")
    run_bash_cmd("python manage.py makemigrations us_covid_api")
    run_bash_cmd("python manage.py migrate")

def generate_drf_spec_schema():
    # generate schema for drf_spectacular
    run_bash_cmd("python manage.py spectacular --file schema.yml")

def print_settings():
    run_bash_cmd("python manage.py print_settings")

In [6]:
'''
    Important functions relating to script main functionality
'''
from tkinter import N
import numpy as np
from math import isnan
from us_covid_api.models import Report, State
from django.core.exceptions import ObjectDoesNotExist
from django.utils.timezone import make_aware

def load_states(state_df: pd.DataFrame):
    def generate_state(state_record):
        state = State(name=state_record['name'], initials=state_record['initials'])
        state.save()
    
    state_df.apply(generate_state, axis = 1)

def load_reports(report_df: pd.DataFrame):
    def generate_report(report_record: pd.Series):
        state = State.objects.get(initials=report_record['state'])

        report_record.loc['state'] = state
        report_record.loc['date'] = make_aware(report_record.loc['date'])
        report_dict = {k:(v if not type(v) is float or not isnan(v) else None) for k, v in report_record.to_dict().items()}

        report_obj = Report(id=None, **report_dict)
        report_obj.save()

    report_df.apply(generate_report, axis=1)

In [7]:
''' Testing code '''

def sample_code():
    df = pd.read_csv('data/all-states-history.csv', parse_dates=['date'])
    df.columns = list(df.columns.map(lambda x: camel_case_to_snake_case(x)))
    record = df.loc[0].copy()
    state = State(name='random', initials=record['state'])
    state.save()
    record.loc['state'] = state
    # record.pop('state')
    value_dict = record.to_dict()
    newdict = {k:(v if not type(v) is float or not isnan(v) else None) for k, v in value_dict.items()}
    print(newdict)
    obj = Report(id=None, **newdict)
    obj.save()
    print('DONE')

In [8]:
'''
    NOTE: this is the main function & entry point of the whole script (most important)
'''

from ast import arg


def run(*args):
    print('script args: ', args)
    # Default behavior is run all below tasks
    if 'no-clear-migrations' not in args: clear_migrations()
    if 'no-clean-cache' not in args: clean_cache()
    if 'no-drop-db' not in args: drop_database()
    if 'no-remigrate-db' not in args: clean_and_remigrate_db()
    if 'no-generate-schema' not in args: generate_drf_spec_schema()
    if 'no-print-settings' not in args: print_settings()
    print('-----DONE PREP-----------------')
    # Data processing part
    df = pd.read_csv('data/all-states-history.csv', parse_dates=['date'])
    df_state = pd.read_json('data/states.json')
    print('Shape df: ', df.shape)
    print('Shape df_state: ', df_state.shape)
    df.columns = list(df.columns.map(lambda x: camel_case_to_snake_case(x)))
    load_states(df_state)
    load_reports(df)
    print('Done importing')

In [9]:
# ''' Notebook test run '''
import threading
import time
NOTEBOOK_ARGS = []

if IS_NOTEBOOK:
    print("Main    : before creating thread")
    x = threading.Thread(target=run, args = NOTEBOOK_ARGS)
    print("Main    : before running thread")
    x.start()
    x.join()
    print("Main    : wait for the thread to finish")
    print("Main    : all done")

Main    : before creating thread
Main    : before running thread
script args:  ()
Exec:  python manage.py clear_cache
Out:  Cache "default" has been cleared!

Exec:  python manage.py clean_pyc
Exec:  pyclean f:\Projects\geo-covid-backend
Exec:  python manage.py flush --noinput
Exec:  python manage.py makemigrations us_covid_api
Out:  Migrations for 'us_covid_api':
  us_covid_api\migrations\0001_initial.py
    - Create model State
    - Create model Report
    - Create index us_covid_ap_date_07cecd_idx on field(s) date of model report
    - Alter unique_together for report (1 constraint(s))

Exec:  python manage.py migrate
Out:  Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, us_covid_api
Running migrations:
This version of djongo does not support "NULL, NOT NULL column validation check" fully. Visit https://nesdis.github.io/djongo/support/
  Applying contenttypes.0001_initial...This version of djongo does not support "schema validation using CONSTRAI

In [10]:
'''
    Testing queries
'''

import timeit
from random import randrange, seed, randint
from datetime import timedelta
import datetime
import threading
import time
# Settings
DAYS_INTERVAL = 5 
seed(0)


# Helper
def random_date(start=datetime.date(2020, 1, 13), end=datetime.date(2021, 3, 7)):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def random_state_initials():
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    return states[randint(0, len(states) - 1)]

def examine_thread_speed(function, args = (), verbose = True):
    if verbose: print('--------------------------')
    # NOTE: there is threading overhead involved
    if args:
        x = threading.Thread(target=function, args = (*args, verbose,))
    else:
        new_func = lambda: function(verbose=verbose)
        x = threading.Thread(target=new_func)
    if verbose: print("Before running thread: ", function.__name__, ", arg: ", args)
    start = timeit.default_timer()
    x.start()
    x.join()
    stop = timeit.default_timer()
    time_diff = stop - start
    if verbose: print('Done, Execution Time (second): ', time_diff)
    return time_diff  


# Tests
def stats_one_day_all_states(date, verbose = True):
    if verbose: print('Date: ', date)
    time_range = (make_aware(datetime.datetime.combine(date, datetime.time.min)),
                make_aware(datetime.datetime.combine(date, datetime.time.max)))
    reports = Report.objects.filter(date__range= time_range)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')

def stats_day_range_all_states(start_day, verbose = True):
    end_day = start_day + datetime.timedelta(days=DAYS_INTERVAL)
    if verbose: print('Start day: ', start_day, ', end day: ', end_day)
    time_range = (make_aware(datetime.datetime.combine(start_day, datetime.time.min)),
                make_aware(datetime.datetime.combine(end_day, datetime.time.max)))
    reports = Report.objects.filter(date__range= time_range)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')
            
def stats_one_state_all_days(state_initials, verbose = True):
    if verbose: print('State: ', state_initials)
    state = State.objects.filter(initials=state_initials).first()
    reports = Report.objects.filter(state_id= state.id)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')
# def stats_few_state_all_days(): pass # ! OUT OF SCOPE


# Results
if IS_NOTEBOOK:
    examine_thread_speed(stats_one_day_all_states, args=(random_date(),));

    start_day = random_date()
    examine_thread_speed(stats_day_range_all_states, args=(start_day,));

    examine_thread_speed(stats_one_state_all_days, args=(random_state_initials(),));

--------------------------
Before running thread:  stats_one_day_all_states , arg:  (datetime.date(2020, 11, 7),)
Date:  2020-11-07
Report retrieved:  56
Done, Execution Time (second):  0.3334782999999675
--------------------------
Before running thread:  stats_day_range_all_states , arg:  (datetime.date(2020, 8, 1),)
Start day:  2020-08-01 , end day:  2020-08-06
Report retrieved:  336
Done, Execution Time (second):  1.307116800000017
--------------------------
Before running thread:  stats_one_state_all_days , arg:  ('NJ',)
State:  NJ
Report retrieved:  392
Done, Execution Time (second):  1.106581800000015


In [11]:

# Helper
def benchmark(function, args_list, inner_verbose=False):
    print('-----------------------------')
    results = []
    for args in args_list:
        time_delta = examine_thread_speed(function, args, verbose=inner_verbose)
        results.append(time_delta)
    print('Run results (second): ', results)
    print('Stats: ')
    display(pd.DataFrame(results).describe())

# Settings
REPEATS = 20
dates = [random_date() for _ in range(REPEATS)]
states_initials_list = [random_state_initials() for _ in range(REPEATS)]

# Results
to_args = lambda list_val: [(x,) for x in list_val]
if IS_NOTEBOOK:
    benchmark(stats_one_day_all_states, to_args(dates))
    benchmark(stats_day_range_all_states, to_args(dates))
    benchmark(stats_one_state_all_days, to_args(states_initials_list))

-----------------------------
Run results (second):  [0.2764097999999535, 0.33654849999999215, 0.27823990000001686, 0.22741439999998647, 0.21016830000002074, 0.14600150000001122, 0.2228284000000258, 0.16245549999996456, 0.11978369999997085, 0.10087370000002238, 0.11942840000000388, 0.10549039999995102, 0.12432609999996203, 0.1460568999999623, 0.13605090000004338, 0.09695579999998927, 0.07394139999996696, 0.1712940999999546, 0.19020209999996496, 0.035654799999974784]
Stats: 


,0
count,20.000000
mean,0.164006
std,0.075827
min,0.035655
25%,0.115944
50%,0.146029
75%,0.213333
max,0.336548


-----------------------------
Run results (second):  [0.5623299999999745, 0.5522054999999568, 0.6231420000000298, 0.3036242999999672, 0.2811393999999723, 0.28353900000001886, 0.3212001999999643, 0.4988066000000231, 0.3528926000000183, 0.3331650999999738, 0.38379829999996673, 0.29514169999998785, 0.28084589999997434, 0.26935880000002044, 0.41379709999995384, 0.29205589999997983, 0.25587399999994886, 0.2911978000000204, 0.3055954000000156, 0.12063790000001973]
Stats: 


,0
count,20.000000
mean,0.351017
std,0.122451
min,0.120638
25%,0.282939
50%,0.304610
75%,0.391298
max,0.623142


-----------------------------
Run results (second):  [0.3081531000000268, 0.2621780000000058, 0.301581399999975, 0.36108319999999594, 0.282807699999978, 0.2790301999999656, 0.2976373000000194, 0.2764819000000216, 0.28072550000001684, 0.2728751000000216, 0.3659511999999836, 0.2624354000000153, 0.2870866000000092, 0.34270040000001245, 0.30604179999994585, 0.29922840000000406, 0.27184640000001536, 0.2788909000000217, 0.26847040000001243, 0.3308319999999867]
Stats: 


,0
count,20.000000
mean,0.296802
std,0.031118
min,0.262178
25%,0.275580
50%,0.284947
75%,0.306570
max,0.365951


In [12]:
if IS_NOTEBOOK:
    run_bash_cmd('jupyter nbconvert --to python .\scripts\load_csv_to_database.ipynb')

Exec:  jupyter nbconvert --to python .\scripts\load_csv_to_database.ipynb
